## Spam Classification Demo <hr>

### 1. Data Acquisition

In [1]:
import pandas as pd

# read in data and reorganize a bit 
spam_data = 'data/uci_ml_spam.txt'
dat = pd.read_csv(spam_data, names=['ham_spam', 'text'], sep='\t')

dat['label'] = dat.ham_spam == 'spam'
dat = dat[['ham_spam', 'label', 'text']]

In [2]:
# inspect the dataset 
print('spam data dimensions: {shape}'.format(shape=dat.shape))
print(dat.head(5))

spam data dimensions: (5572, 3)
  ham_spam  label                                               text
0      ham  False  Go until jurong point, crazy.. Available only ...
1      ham  False                      Ok lar... Joking wif u oni...
2     spam   True  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  False  U dun say so early hor... U c already then say...
4      ham  False  Nah I don't think he goes to usf, he lives aro...


In [4]:
print('distribution of labels:')
dat.ham_spam.value_counts() / sum(dat.ham_spam.value_counts())

distribution of labels:


ham     0.865937
spam    0.134063
Name: ham_spam, dtype: float64

In [5]:
from random import sample

# test-train split 
train_idx = sample(range(len(dat)), int(len(dat)*.75))
test_idx = [n for n in set(range(len(dat))) - set(train_idx)]

train_text = list(dat.text.iloc[train_idx])
train_labs = list(dat.label.iloc[train_idx])

test_text = list(dat.text.iloc[test_idx])
test_labs = list(dat.label.iloc[test_idx])


In [6]:
len(train_text)

4179

### 2. Preprocessing

In [19]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# preprocess text (cast to a count-based document-term matrix)
vectorizer = CountVectorizer(lowercase=True,
                             ngram_range=(1,1),
                             stop_words='english')
train_dtm = vectorizer.fit_transform(train_text)

# check out the transformed data 
# vectorizer.get_feature_names()[999:1011]
# list(vectorizer.get_stop_words())[0:10]
# vectorizer.get_params()
# pd.DataFrame(train_dtm.A, columns=vectorizer.get_feature_names()).to_string())

### Model Training

In [20]:
from sklearn.naive_bayes import MultinomialNB

# model training 
clf = MultinomialNB()
clf.fit(train_dtm, train_labs)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Model Evaluation

In [21]:
# model evaluation 
test_dtm = vectorizer.transform(test_text)

predictions = clf.predict(test_dtm)

In [22]:
from sklearn import metrics

accuracy = metrics.accuracy_score(test_labs, predictions)
print(f'out of the box accuracy: {round(accuracy,3)} (waow)')

out of the box accuracy: 0.985 (waow)


In [23]:
eval_df = pd.DataFrame([*zip(test_labs, predictions, test_text)], 
                       columns=('label', 'prediction', 'text'))

print(eval_df.head(5))

   label  prediction                                               text
0  False       False  U dun say so early hor... U c already then say...
1  False       False  Nah I don't think he goes to usf, he lives aro...
2  False       False  As per your request 'Melle Melle (Oru Minnamin...
3   True        True  WINNER!! As a valued network customer you have...
4  False       False  I've been searching for the right words to tha...


In [25]:
# can also add `margins=True` 
conf_mat = pd.crosstab(eval_df.label, eval_df.prediction, 
                       rownames=['observed'], colnames=['predicted'])

print(conf_mat)

predicted  False  True 
observed               
False       1196      8
True          13    176


In [26]:
# we can also view a confusion matrix with column-normalized values
print(round(conf_mat / conf_mat.sum(axis=0), 3))

predicted  False  True 
observed               
False      0.989  0.043
True       0.011  0.957


In [27]:
# look at the bad predictions 
print(eval_df[eval_df.label != eval_df.prediction].sort_values('label'))

      label  prediction                                               text
105   False        True   Yes:)from last week itself i'm taking live call.
1186  False        True  I (Career Tel) have added u as a contact on IN...
244   False        True                                       26th OF JULY
331   False        True  Hey...Great deal...Farm tour 9am to 5pm $95/pa...
1164  False        True            Have you laid your airtel line to rest?
1154  False        True                            Gettin rdy to ship comp
1103  False        True             Mathews or tait or edwards or anderson
966   False        True                  Unlimited texts. Limited minutes.
1075   True       False  thesmszone.com lets you send free anonymous an...
1069   True       False  Block Breaker now comes in deluxe format with ...
769    True       False  Hi babe its Jordan, how r u? Im home from abro...
737    True       False   You have 1 new message. Please call 08712400200.
706    True       False  